In [1]:
# import
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
import os

In [2]:
# label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

# def class_2_type(root):
#     label = ""
#     if "雙踝" in root:
#         label = "0"
#     elif "三踝" in root:
#         label = "1"
#     return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================

In [3]:
def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type   

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            if label != "":
                dataset.append(
                                {   
                                    'uuid': root.split("\\")[-1],
                                    'label': label,
                                    'image_path': os.path.join(root, file)
                                }
                            )

    return dataset

In [4]:
## 參數設置
image_dir = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"
concat_type = "voting"
class_count = 3
maru_part=None

side_pred = ""
front_pred = ""

In [5]:

## load data and  labels
# =========================
data = load_path(image_dir, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)
# =========================


## split image
# =========================
train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=44, stratify=images['Label'])
print("Training set label distribution:\n", train_df['Label'].value_counts(normalize=False))
print("Test set label distribution:\n", test_df['Label'].value_counts(normalize=False))

# 關閉翻轉
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
                                                                    validation_split=0.2)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
# =========================

Training set label distribution:
 0    128
2     95
1     93
Name: Label, dtype: int64
Test set label distribution:
 0    32
1    24
2    24
Name: Label, dtype: int64


In [ ]:
## train front and side
save_path_arr = ["front", "side"]
for save_path in save_path_arr:
    
    # load model
    # =========================
    pretrained_model = tf.keras.applications.resnet50.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')

    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
    x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

    outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
    model = tf.keras.Model(inputs, outputs)
    # print(model.summary())
    # =========================

    ## 分資料
    # =========================
    if save_path == "side":
        train_df.loc[:, "Filepath"] = train_df["Filepath"].str.replace("front", "side")
        test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("front", "side")
    # 確認
    print(train_df.iloc[0]['Filepath'].split("\\")[-3])
    
    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=False,
        seed=42,
        subset='training'
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=False,
        seed=42,
        subset='validation'
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    # =========================

    
    ## compile and evaluate
    # =========================

    print("-------Training " + save_path + "_" + concat_type + "-------")
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    ## early stop 
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    history=model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
    ## no early stop
    # history = model.fit(train_images, validation_data=val_images, epochs=30)

    results = model.evaluate(test_images, verbose=0)
    # =========================


    ## save model to this path
    # =========================
    model.save("./weights/"+concat_type+"_"+save_path + "_frac.h5")
    # =========================


    ## print results
    # =========================
    # print(save_path + "_" + concat_type + "_Results:")
    pred = model.predict(test_images)

    ## 把pred分開存
    if save_path == "front":
        front_pred = pred
    else:
        side_pred = pred
    predicted_labels = np.argmax(pred, axis=1)
    # f1 = f1_score(test_images.labels, predicted_labels, average='macro')
    # precision = precision_score(test_images.labels, predicted_labels, average='macro')
    # recall = recall_score(test_images.labels, predicted_labels, average='macro')

    # print(results)
    # print(f"Test Accuracy: {np.round(results[1], 2)}")
    # print(f"f1 score: {np.round(f1, 2)}")
    # print(f"precision: {np.round(precision, 2)}")
    # print(f"recall: {np.round(recall, 2)}")
    # =========================


    # create plots for accuracy and save it
    # =========================
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    figAcc = plt.gcf()
    my_file = os.path.join("./plots/"+concat_type+"_"+save_path+"_Accuracy.jpeg")
    figAcc.savefig(my_file)
    plt.clf()
    # =========================


    ## create plots for loss and save it
    # =========================
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    figAcc = plt.gcf()
    my_file = os.path.join("./plots/"+concat_type+"_"+save_path+"_Loss.jpeg")
    figAcc.savefig(my_file)
    plt.clf()
    # =========================


    ## plot confusion matrix
    # =========================
    if class_count == 2:
        display_labels = [0, 1]
    elif class_count == 3:
        display_labels = [0, 1, 2]
    elif class_count == 4:
        display_labels = [0, 1, 2, 3]


    cm = confusion_matrix(test_images.labels, predicted_labels)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = display_labels)
    cm_display.plot()
    plt.title('Confusion Matrix')
    figAcc = plt.gcf()
    my_file = os.path.join("./plots/"+concat_type+"_"+save_path+"_Confusion Matrix.jpeg")
    figAcc.savefig(my_file)
    plt.clf()
    # =========================

front
Found 253 validated image filenames belonging to 3 classes.
Found 63 validated image filenames belonging to 3 classes.
Found 80 validated image filenames belonging to 3 classes.
-------Training front_voting-------
Epoch 1/30
4/4 [==============================] - 5s 728ms/step - loss: 1.4417 - accuracy: 0.3439 - val_loss: 1.1214 - val_accuracy: 0.4286
Epoch 2/30
4/4 [==============================] - 1s 229ms/step - loss: 1.0681 - accuracy: 0.4506 - val_loss: 1.0391 - val_accuracy: 0.5556
Epoch 3/30
4/4 [==============================] - 1s 229ms/step - loss: 0.9873 - accuracy: 0.5257 - val_loss: 1.0066 - val_accuracy: 0.5873
Epoch 4/30
4/4 [==============================] - 1s 235ms/step - loss: 0.9205 - accuracy: 0.6047 - val_loss: 0.9154 - val_accuracy: 0.6190
Epoch 5/30
4/4 [==============================] - 1s 230ms/step - loss: 0.8419 - accuracy: 0.6640 - val_loss: 0.8350 - val_accuracy: 0.7143
Epoch 6/30
4/4 [==============================] - 1s 230ms/step - loss: 0.7782 -

d:\anaconda3\envs\bone_20240719\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
d:\anaconda3\envs\bone_20240719\lib\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


side
Found 253 validated image filenames belonging to 3 classes.
Found 63 validated image filenames belonging to 3 classes.
Found 80 validated image filenames belonging to 3 classes.
-------Training side_voting-------
Epoch 1/30
4/4 [==============================] - 3s 503ms/step - loss: 1.1862 - accuracy: 0.3320 - val_loss: 1.0711 - val_accuracy: 0.3968
Epoch 2/30
4/4 [==============================] - 1s 309ms/step - loss: 1.0689 - accuracy: 0.4862 - val_loss: 1.0576 - val_accuracy: 0.4921
Epoch 3/30
4/4 [==============================] - 1s 283ms/step - loss: 1.0139 - accuracy: 0.5534 - val_loss: 0.9874 - val_accuracy: 0.5714
Epoch 4/30
4/4 [==============================] - 1s 306ms/step - loss: 0.9427 - accuracy: 0.5850 - val_loss: 0.9458 - val_accuracy: 0.6508
Epoch 5/30
4/4 [==============================] - 1s 306ms/step - loss: 0.8998 - accuracy: 0.6047 - val_loss: 0.9132 - val_accuracy: 0.6190
Epoch 6/30
4/4 [==============================] - 1s 298ms/step - loss: 0.8441 - a

d:\anaconda3\envs\bone_20240719\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [7]:
from scipy.stats import mode
from sklearn.metrics import accuracy_score

vote_type_arr = ["front", "side", "soft", "hard", "product"]
for vote_type in vote_type_arr:
    print("------- "+vote_type+" -------")
    if vote_type == "front":
        predicted_labels = np.argmax(front_pred, axis=1)
    if vote_type == "side":
        predicted_labels = np.argmax(side_pred, axis=1)
    if vote_type == "soft":
        soft_pred = np.argmax((front_pred + side_pred) / 2 , axis=1)
        predicted_labels = soft_pred
    if vote_type == "hard":
        hard_front_pred = np.argmax(front_pred, axis=1)  
        hard_side_pred = np.argmax(side_pred, axis=1)  
        hard_pred = mode([hard_front_pred, hard_side_pred], axis=0).mode[0]  
        predicted_labels = hard_pred
    if vote_type == "product":
        product_pred = [[a * b for a, b in zip(sublist1, sublist2)] for sublist1, sublist2 in zip(front_pred, side_pred)]
        product_pred = np.argmax(product_pred, axis=1) 
        predicted_labels = product_pred




    
    acc = accuracy_score(test_images.labels, predicted_labels)
    f1 = f1_score(test_images.labels, predicted_labels, average='macro')
    precision = precision_score(test_images.labels, predicted_labels, average='macro')
    recall = recall_score(test_images.labels, predicted_labels, average='macro')

    print(f"Test Accuracy: {np.round(acc, 2)}")
    print(f"f1 score: {np.round(f1, 2)}")
    print(f"precision: {np.round(precision, 2)}")
    print(f"recall: {np.round(recall, 2)}")


------- front -------
Test Accuracy: 0.79
f1 score: 0.76
precision: 0.76
recall: 0.76
------- side -------
Test Accuracy: 0.66
f1 score: 0.61
precision: 0.61
recall: 0.62
------- soft -------
Test Accuracy: 0.76
f1 score: 0.73
precision: 0.73
recall: 0.74
------- hard -------
Test Accuracy: 0.65
f1 score: 0.6
precision: 0.62
recall: 0.61
------- product -------
Test Accuracy: 0.76
f1 score: 0.73
precision: 0.73
recall: 0.74


In [8]:
# 錯的
print(np.where(test_images.labels != hard_front_pred)[0])
print(np.where(test_images.labels != hard_side_pred)[0])
print(np.where(test_images.labels != soft_pred)[0])
print(np.where(test_images.labels != hard_pred)[0])
print(np.where(test_images.labels != product_pred)[0])


[11 15 17 20 29 32 37 40 43 44 53 65 66 67 68 71 78]
[ 4  6 10 11 19 20 22 25 26 29 32 34 35 40 41 43 50 53 57 60 61 63 65 66
 68 69 76]
[11 15 17 19 20 29 32 35 40 41 43 44 53 65 66 67 68 71 78]
[ 4  6 10 11 15 17 19 20 22 25 26 29 32 34 35 37 40 43 44 50 53 57 63 65
 66 67 68 76]
[11 15 17 19 20 29 32 35 40 41 43 44 53 65 66 67 68 71 78]


In [9]:
# test_df["pred"] = predicted_labels

In [10]:
# test_df

## grad_cam


In [11]:
from keras.preprocessing import image
import matplotlib.cm as cm

def make_gradcam_heatmap(img_array, model, last_conv_layer_name="conv5_block3_out", pred_index=None):
    # 建立一個模型，同時輸出最後一個卷積層和整個模型的預測結果
    grad_model = tf.keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # 計算對於輸入圖像的預測類別，相對於最後一個卷積層的梯度
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # 輸出分類神經元相對於最後一個卷積層的輸出特徵圖的梯度
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 這是一個向量，其中每個數字都是特定特徵圖通道上的梯度的平均強度
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 將特徵圖乘以權重，等於該特徵圖中的某些區域對於該分類的重要性
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap) # 然後將所有通道相加以獲得熱圖

    # 為了視覺化，將熱圖正規化0~1之間
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, alpha=0.4):
    # 載入原始圖像
    img = tf.keras.utils.load_img(img_path)
    img = tf.keras.utils.img_to_array(img)

    # 將熱圖重新縮放到0-255的範圍
    heatmap = np.uint8(255 * heatmap)

    # 使用Jet色彩映射將熱圖上色
    jet = cm.get_cmap("Purples")

    # 使用Jet色彩映射的RGB值
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # 創建帶有RGB色彩的熱圖圖像
    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    # 在原始圖像上疊加熱圖
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.utils.array_to_img(superimposed_img)
    plt.imshow(superimposed_img)
    plt.axis('off')  # 不顯示坐標軸
    plt.show()

In [12]:
# import matplotlib.image as mpimg
# chosen_model = "frac.h5"

# filtered_df = test_df[(test_df['Label'] == "1") & (test_df['pred'] == 2)]

# img = filtered_df["Filepath"].values
# for im in img:
#     temp_img = image.load_img(im, target_size=(224, 224))
#     x = image.img_to_array(temp_img)
#     x = np.expand_dims(x, axis=0)
#     images = np.vstack([x])
#     heatmap = make_gradcam_heatmap(images, tf.keras.models.load_model(chosen_model))
#     print(f"image path={im}")
#     save_and_display_gradcam(im, heatmap)
#     print("##################################################################################")
# ############################

In [13]:
## 單一查看
# import matplotlib.image as mpimg
# chosen_model = ".//weights//中榮//front//transfer_imagenet//all//imagenet_AP+Mortise_swift_cut_2class.h5"

# im = "E://data_bone//1-swift_cut//front//三踝//001744332C_R.jpg"

# temp_img = image.load_img(im, target_size=(224, 224))
# x = image.img_to_array(temp_img)
# x = np.expand_dims(x, axis=0)
# images = np.vstack([x])
# heatmap = make_gradcam_heatmap(images, tf.keras.models.load_model(chosen_model))
# print(f"image path={im}")
# save_and_display_gradcam(im, heatmap)
# print("##################################################################################")